# MOEX Minute Loader Demo

Используем свежий модуль `moex_ohcl_loader` для автоматической докачки минутных свечей с Московской биржи по тикерам, встречающимся в последнем тесте из файла `SiBDt.csv`, а затем построим эквити с помощью уже существующего конструктора.

In [1]:
from pathlib import Path

import pandas as pd

from src.data.moex_ohcl_loader import download_moex_ohlc
from src.data.equity_builder import build_equity

In [2]:
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

log_path = Path("trade_data") / "SiBDt.csv"
candles = download_moex_ohlc(log_path, save=True)
# candles = download_moex_ohlc(log_path)
candles.head()

,DATE_TIME,OPEN,HIGH,LOW,CLOSE,VOL,TICKER,1_step_price
0,2025-07-23 09:22:00,80671.0,80682.0,80667.0,80682.0,56,SiU5,1.0
1,2025-07-23 09:23:00,80684.0,80749.0,80681.0,80746.0,826,SiU5,1.0
2,2025-07-23 09:24:00,80746.0,80828.0,80746.0,80810.0,2653,SiU5,1.0
3,2025-07-23 09:25:00,80811.0,80944.0,80810.0,80934.0,3324,SiU5,1.0
4,2025-07-23 09:26:00,80934.0,80981.0,80933.0,80933.0,3330,SiU5,1.0


In [ ]:
candles_path = Path("OHCL") / f"{log_path.stem}_ohcl.csv"
candles_path

In [ ]:
equity = build_equity(candles_csv=candles_path, trades_csv=log_path)
equity.head()

In [ ]:
equity.tail()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scattergl(        # ⚡ ScatterGL — рендер на GPU
    x=equity['DATE_TIME'],
    y=equity['Equity'],
    mode='lines',
    line=dict(width=1, color='blue')
))
fig.show()

In [ ]:
# 🎯 Финальный отчёт по Equity
from src.data.equity_report import build_equity_report

equity_report = build_equity_report(equity)
print(equity_report.to_pretty_text())